In [1]:
import dlib
import cv2
import numpy as np
import os
import pickle
import time
import matplotlib.pyplot as plt # Optional, for debugging specific frames if needed

# --- 1. Load Dlib Models (Local Paths) ---
DLIB_MODELS_PATH = r"C:\Users\Wiam\Desktop\AI_Guard\Dlib_face_recognition" # Current directory, assuming .dat files are here
# Or specify a subfolder if you put them there, e.g., "models/"
landmark_predictor_file = "shape_predictor_68_face_landmarks.dat"
face_recognition_model_file = "dlib_face_recognition_resnet_model_v1.dat"

landmark_predictor_path = os.path.join(DLIB_MODELS_PATH, landmark_predictor_file)
face_recognition_model_path = os.path.join(DLIB_MODELS_PATH, face_recognition_model_file)

if not (os.path.exists(landmark_predictor_path) and os.path.exists(face_recognition_model_path)):
    print(f"ERROR: Dlib model files not found. Searched in '{os.path.abspath(DLIB_MODELS_PATH)}'")
    print(f"Ensure '{landmark_predictor_file}' and '{face_recognition_model_file}' are in that location.")
    exit()

try:
    print("Loading Dlib models...")
    face_detector = dlib.get_frontal_face_detector() 
    landmark_predictor = dlib.shape_predictor(landmark_predictor_path)
    face_encoder = dlib.face_recognition_model_v1(face_recognition_model_path)
    print("Dlib models loaded successfully.")
except Exception as e:
    print(f"Error loading dlib models: {e}")
    exit()

# --- 2. Load Enrolled Data (Local Path) ---
ENROLLMENT_FILE_PATH = r"C:\Users\Wiam\Desktop\AI_Guard\ai_guard_GENERATED_enrollments.pkl" # Assuming it's in the same directory

if not os.path.exists(ENROLLMENT_FILE_PATH):
    print(f"ERROR: Enrollment file not found at '{os.path.abspath(ENROLLMENT_FILE_PATH)}'")
    exit()
try:
    with open(ENROLLMENT_FILE_PATH, "rb") as f:
        enrollment_data = pickle.load(f)
    known_embeddings_db = enrollment_data["embeddings"] 
    known_labels_db = enrollment_data["labels"]         
    print(f"Loaded {known_embeddings_db.shape[0]} enrolled embeddings for {len(set(known_labels_db))} unique individuals.")
    if known_embeddings_db.shape[0] == 0:
        print("ERROR: Enrollment database is empty!")
        exit()
except Exception as e:
    print(f"Error loading enrollment data: {e}")
    exit()

# --- 3. Recognition Parameters ---
RECOGNITION_THRESHOLD = 0.55  # Start with this, TUNE IT!
DLIB_UPSAMPLE_DETECT_REALTIME = 0 # Start with 0 for speed. Increase to 1 if missing faces.
DLIB_JITTERS_ENCODE_REALTIME = 1  # Fewer jitters for speed.

# --- 4. Alarm System ---
last_alarm_time = 0
ALARM_COOLDOWN_SECONDS = 5 # Cooldown for alarms
alarm_active = False # State to keep alarm visually on for a bit

# def play_alarm_sound(): # Optional
#     try:
#         from playsound import playsound
#         # Make sure you have an alarm sound file (e.g., alarm.wav)
#         # playsound("alarm.wav") # Or use system beeps
#         print("\007") # System beep (might not work on all systems/terminals)
#     except Exception as e:
#         print(f"Could not play sound: {e}")

def trigger_alarm(frame_to_save=None, unknown_person_count=0, recognized_person_on_watchlist=None):
    global last_alarm_time, alarm_active
    current_time = time.time()
    
    trigger_condition_met = False
    reason = ""

    if unknown_person_count > 0:
        trigger_condition_met = True
        reason = f"{unknown_person_count} Unknown person(s) detected!"
    elif recognized_person_on_watchlist:
        trigger_condition_met = True
        reason = f"WATCHLIST ALERT: {recognized_person_on_watchlist} detected!"

    if trigger_condition_met:
        if (current_time - last_alarm_time) > ALARM_COOLDOWN_SECONDS:
            print(f"--- ALARM TRIGGERED ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---")
            print(f"  Reason: {reason}")
            # play_alarm_sound() # Optional
            alarm_active = True # Set alarm state
            last_alarm_time = current_time
            
            if frame_to_save is not None:
                save_dir = "alarm_snapshots"
                os.makedirs(save_dir, exist_ok=True)
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                alarm_image_path = os.path.join(save_dir, f"alarm_event_{timestamp}.jpg")
                cv2.imwrite(alarm_image_path, frame_to_save)
                print(f"  Snapshot saved to: {alarm_image_path}")
        return True # Indicates an alarm condition is active (new or recent)
    
    # Reset alarm_active visual state if cooldown has passed since it was set
    if alarm_active and (current_time - last_alarm_time) > ALARM_COOLDOWN_SECONDS / 2: # Keep visual for half cooldown
        alarm_active = False
        
    return False # No new alarm, or on cooldown for a new trigger

# --- 5. Face Recognition Function for a Single Frame ---
def recognize_faces_in_frame(frame, known_embeddings, known_labels):
    # Frame should be BGR from OpenCV VideoCapture
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    detections = face_detector(img_rgb, DLIB_UPSAMPLE_DETECT_REALTIME)
    recognized_people_in_frame = [] 

    for d_face in detections:
        shape = landmark_predictor(img_rgb, d_face)
        unknown_embedding = np.array(face_encoder.compute_face_descriptor(img_rgb, shape, DLIB_JITTERS_ENCODE_REALTIME))
        distances = np.linalg.norm(known_embeddings - unknown_embedding, axis=1)
        min_distance_idx = np.argmin(distances)
        min_distance_val = distances[min_distance_idx]

        name_recognized = "Unknown"
        if min_distance_val < RECOGNITION_THRESHOLD:
            name_recognized = known_labels[min_distance_idx]
        
        x1, y1, x2, y2 = d_face.left(), d_face.top(), d_face.right(), d_face.bottom()
        recognized_people_in_frame.append(((x1, y1, x2, y2), name_recognized, min_distance_val))
        
    return recognized_people_in_frame

# --- 6. Main Real-Time Loop ---
# WATCHLIST_NAMES = {"Some_Known_Person_To_Alert_For"} # Example, use actual names from your labels
WATCHLIST_NAMES = set() # Empty watchlist for now

video_source = 0 # 0 for default webcam. Change if you have multiple or use a video file path.
cap = cv2.VideoCapture(video_source)

if not cap.isOpened():
    print(f"Error: Could not open video source: {video_source}")
    exit()

print(f"\nStarting AI Guard System. Press 'q' to quit.")
print(f"Using recognition threshold: {RECOGNITION_THRESHOLD}")
print(f"Webcam/Video FPS: {cap.get(cv2.CAP_PROP_FPS)}") # May not be accurate for all webcams
print(f"Webcam/Video Resolution: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")


frame_processing_interval = 3 # Process every Nth frame
frame_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    frame_counter += 1
    display_frame = frame.copy() # Work on a copy for display
    alarm_triggered_for_this_cycle = False
    
    if frame_counter % frame_processing_interval == 0: # Process this frame
        recognition_results = recognize_faces_in_frame(frame, known_embeddings_db, known_labels_db)
        
        unknowns_this_frame = 0
        person_on_watchlist_detected = None

        for (box, name, distance) in recognition_results:
            x1, y1, x2, y2 = box
            color = (0, 255, 0) # Green for known
            
            if name == "Unknown":
                color = (0, 0, 255) # Red for unknown
                unknowns_this_frame += 1
            elif name in WATCHLIST_NAMES:
                color = (0, 165, 255) # Orange for watchlist
                person_on_watchlist_detected = name
            
            cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
            label_text = f"{name} ({distance:.2f})"
            cv2.putText(display_frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if unknowns_this_frame > 0 or person_on_watchlist_detected:
            alarm_triggered_for_this_cycle = trigger_alarm(
                frame_to_save=frame, # Save original frame for alarm
                unknown_person_count=unknowns_this_frame,
                recognized_person_on_watchlist=person_on_watchlist_detected
            )

    # Visual indicator for active alarm state (even if on cooldown for sound/new log)
    if alarm_active: # 'alarm_active' is set by trigger_alarm
        cv2.putText(display_frame, "ALARM ACTIVE", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    else: # Check and potentially reset alarm_active if cooldown passed (handled in trigger_alarm too)
         # but this explicit call makes sure it's reset if no new trigger happens
        current_time_for_reset = time.time()
        if (current_time_for_reset - last_alarm_time) > ALARM_COOLDOWN_SECONDS :
             alarm_active = False


    cv2.imshow('AI Guard - Live Surveillance', display_frame)

    key = cv2.waitKey(1) & 0xFF # Wait for 1 ms
    if key == ord('q'):
        print("Quitting...")
        break
    elif key == ord('a'): # Manual alarm trigger for testing
        print("Manual alarm test!")
        trigger_alarm(frame_to_save=frame, unknown_person_count=1)


cap.release()
cv2.destroyAllWindows()
print("AI Guard System stopped.")

Loading Dlib models...
Dlib models loaded successfully.
Loaded 38028 enrolled embeddings for 5661 unique individuals.

Starting AI Guard System. Press 'q' to quit.
Using recognition threshold: 0.55
Webcam/Video FPS: 30.0
Webcam/Video Resolution: 640x480
Quitting...
AI Guard System stopped.
